In [5]:
import gymnasium as gym
import numpy as np
import torch.nn as nn
import torch.optim as optim

from experiments.utils import init_weights, make_var, print_model_info, save_img
from miniworld.wrappers import PyTorchObsWrapper

from tqdm import tqdm

In [4]:

batch_size = 128

buffer = np.zeros(shape=(32768, 3, 80, 60))
cur_gen_idx = 0
idx_avail = 0


def gen_data():
    global cur_gen_idx
    global idx_avail

    for _ in range(32):
        buffer[cur_gen_idx] = env.reset()
        cur_gen_idx = (cur_gen_idx + 1) % buffer.shape[0]
        idx_avail = max(idx_avail, cur_gen_idx)

In [6]:

# parser = argparse.ArgumentParser()
# parser.add_argument('--map-name', required=True)
# args = parser.parse_args()

env = gym.make("MiniWorld-Hallway-v0")
env.domain_rand = True
env = PyTorchObsWrapper(env)

model = Model()
model.cuda()
print_model_info(model)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

while idx_avail <= batch_size:
    gen_data()

for i in tqdm(range(10000000)):
    print(i, cur_gen_idx)

    batch_idx = np.random.randint(0, idx_avail - batch_size)
    batch = buffer[batch_idx : (batch_idx + batch_size)]
    batch = make_var(batch)

    y = model(batch)

    # Generate data while the GPU is computing
    gen_data()

    optimizer.zero_grad()
    diff = y - batch
    loss = (diff * diff).mean()  # L2 loss
    # loss = (y - batch).abs().mean()
    loss.backward()
    optimizer.step()

    print(loss.data.item())


Model(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(32, 32, kernel_size=(5, 5), stride=(2, 2))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2))
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Dropout(p=0.2, inplace=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(32, 32, kernel_size=(6, 6), stride=(2, 2))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose2d(32, 32, kernel_size=(6, 6), stride=(2, 2))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose2d(32, 32, kernel_size=(4, 4), stride=(2, 2))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.